In [1]:
import numpy as np
def feature_sign_search(base_vectors, input_vect, gamma):
    effective_zero = 1e-18
    
    Atrans_A = np.dot(base_vectors.T,base_vectors)
    Atrans_ip = np.dot(base_vectors.T, input_vect)
    
    #coef_vect is the coefficient vector
    coef_vect = np.zeros(Atrans_A.shape[0])
    #Holds sign of the coef_vect vector.
    signs = np.zeros(Atrans_A.shape[0], dtype=np.int8)
    active_set = set()
    zero_opt = np.inf
    nzero_opt = 0
       
    #Second term becomes zero initially hence is not important.
    grad = - 2 * Atrans_ip + 2 * np.dot(Atrans_A, coef_vect)

    #Compute cost function for line search
    sds = np.dot(input_vect.T, input_vect)
    while not(zero_opt <= gamma and np.allclose(nzero_opt, 0)):
        if np.allclose(nzero_opt, 0):
            candidate = np.argmax(np.abs(grad) * (signs == 0))

            if grad[candidate] > gamma:
                signs[candidate] = -1.
                coef_vect[candidate] = 0.

                active_set.add(candidate)
            elif grad[candidate] < -gamma:
                signs[candidate] = 1.
                coef_vect[candidate] = 0.

                active_set.add(candidate)
            if len(active_set) == 0:
                break
        indices = np.array(sorted(active_set))
        sub_gram = Atrans_A[np.ix_(indices, indices)]
        sub_corr = Atrans_ip[indices]
        sub_sign = signs[indices]
        rhs = sub_corr - gamma * sub_sign / 2
        
        new_coef_vect = np.linalg.solve(sub_gram, rhs)
        new_signs = np.sign(new_coef_vect)
        sub_oldsol = coef_vect[indices]
        sign_flips = np.where(abs(new_signs - sub_sign) > 1)[0]
        if len(sign_flips) > 0:
            best_obj = np.inf
            best_curr = None
            best_curr = new_coef_vect
            best_obj = (sds + (np.dot(new_coef_vect,
                                      np.dot(sub_gram, new_coef_vect))
                        - 2 * np.dot(new_coef_vect, sub_corr))
                        + gamma * abs(new_coef_vect).sum())            
    
            for idx in sign_flips:
                a = new_coef_vect[idx]
                b = sub_oldsol[idx]
                prop = b / (b - a)
                curr = sub_oldsol - prop * (sub_oldsol - new_coef_vect)
                cost = sds + (np.dot(curr, np.dot(sub_gram, curr))
                              - 2 * np.dot(curr, sub_corr)
                              + gamma * abs(curr).sum())

                if cost < best_obj:
                    best_obj = cost
                    best_curr = curr

        else:
            best_curr = new_coef_vect
        coef_vect[indices] = best_curr
        zeros = indices[np.abs(coef_vect[indices]) < effective_zero]
        coef_vect[zeros] = 0
        
        active_set.difference_update(zeros)
        grad = - 2 * Atrans_ip + 2 * np.dot(Atrans_A, coef_vect)
        zero_opt = np.max(abs(grad[signs == 0]))
        nzero_opt = np.max(abs(grad[signs != 0] + gamma * signs[signs != 0]))
    return coef_vect

In [3]:
A=np.random.random((10,10))
y=np.random.random(10)
gamma=[0.12,0.2,0.5,1,2,4,8,10,20,50,100]
for g in gamma:
    #Predicted coefficient vector
    X_predicted=feature_sign_search(A,y,g)
    #Actual Coefficient vector
    X=np.linalg.solve(A,y)

    print("For gamma ={0}, norm of error is:{1}".format(g,(np.linalg.norm(X-X_predicted))))
    print("Corresponding Coefficient Vector:{0}".format(X_predicted))
    print()

""" Run the code a few times if it hangs by restarting the kernel  """

For gamma =0.12, norm of error is:4.465693537158788
Corresponding Coefficient Vector:[-0.06821536  0.00313132  0.          0.          0.62771135 -0.04587954
  0.          0.22175316  0.         -0.24083373]

For gamma =0.2, norm of error is:4.504056689297736
Corresponding Coefficient Vector:[ 0.          0.          0.          0.          0.57674627  0.          0.
  0.15629456  0.         -0.19507784]

For gamma =0.5, norm of error is:4.568856147339957
Corresponding Coefficient Vector:[ 0.          0.          0.00176084  0.          0.53237512  0.          0.
  0.05727834  0.          0.        ]

For gamma =1, norm of error is:4.567650486235373
Corresponding Coefficient Vector:[ 0.          0.          0.          0.          0.45299596  0.          0.
  0.06566718  0.          0.        ]

For gamma =2, norm of error is:4.569718755501085
Corresponding Coefficient Vector:[ 0.          0.          0.          0.          0.292849    0.          0.
  0.08118873  0.          0.      

' Run the code a few times if it hangs by restarting the kernel  '